In [214]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [145]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [57]:
# Prelabeled data from Kaggle
train_df = pd.read_csv('sentiment_dataset.csv', delimiter=',', encoding='ISO-8859-1')
train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [144]:
train_df['sentiment'] = train_df['sentiment'].replace({'neutral' : 2, 'negative' : 0, 'positive' : 4})
train_df = train_df[['text', 'sentiment']]
train_df.head()

,text,sentiment
0,"I`d have responded, if I were going",2
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0


In [143]:
# Preprocessing the text data
joined_text = []
stemmer = LancasterStemmer()
stopwords_punc = list(stopwords.words('english'))+list(punctuation)
for i in train_df.index:
    text = re.sub('[^a-zA-Z]', ' ', str(train_df['text'][i]))
    text = text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    lem = SnowballStemmer("english")
    text = [lem.stem(word) for word in text if not word in stopwords_punc] 
    joined_text.append(" ".join(text))
train_clean = pd.DataFrame({'text':joined_text,'sentiment':train_df['sentiment']})

In [119]:
X = train_clean['text']
y = train_clean['sentiment']

In [146]:
tfidf = TfidfVectorizer(norm = "l2")
tfidf.fit(X)
Xt = tfidf.transform(X)
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state= 4)

In [147]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [148]:
y_pred = model.predict(X_test)

In [149]:
np.mean(y_pred == np.array(y_test))

0.6818264507913407

In [227]:
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", #"Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost", "Logistic Regression","Naive Bayes",
#"QDA"
    ]


classifiers = [
    KNeighborsClassifier(2),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    LogisticRegression(max_iter=1000),
    GaussianNB()
#     QuadraticDiscriminantAnalysis()
]

# X, y = X_data_reshape, y_data

# X = StandardScaler().fit_transform(X)
# X_train, X_test, y_train, y_test = \
#     train_test_split(X, y, test_size=.2)

tfidf = TfidfVectorizer(norm = "l2")
tfidf.fit(X)
Xt = tfidf.transform(X)
Xt_dense = Xt.toarray()
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state= 4)

# TODO (Apply): All cross-validation

max_score = 0.0
max_class = ''
# iterate over classifiers
for name, clf in zip(names, classifiers):
    start_time = time.time()
#     clf.fit(X_train, y_train)
#     score = 100.0 * clf.score(X_test, y_test)

    # Do Cross Validation
    if name == "Naive Bayes" or name == "QDA":
        # For GaussianNB, use the dense data
        scores = cross_val_score(clf, Xt_dense, y, cv=5)
    else:
        scores = cross_val_score(clf, Xt, y, cv=5)
    
    score = 100.0 * np.mean(scores)
    
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time.time() - start_time))
    
    if score > max_score:
        clf_best = clf
        max_score = score
        max_class = name

print(80*'-' )
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))
#plot the output of the various algorithms

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

Classifier = Nearest Neighbors, Score (test, accuracy) = 45.92, Training time = 6.80 seconds
Classifier = Linear SVM, Score (test, accuracy) = 49.55, Training time = 163.10 seconds
Classifier = RBF SVM, Score (test, accuracy) = 66.37, Training time = 420.00 seconds
Classifier = Decision Tree, Score (test, accuracy) = 52.12, Training time = 1.21 seconds
Classifier = Random Forest, Score (test, accuracy) = 40.47, Training time = 0.18 seconds
Classifier = Neural Net, Score (test, accuracy) = 67.17, Training time = 2087.92 seconds
Classifier = AdaBoost, Score (test, accuracy) = 67.08, Training time = 48.60 seconds
Classifier = Logistic Regression, Score (test, accuracy) = 69.08, Training time = 12.16 seconds
Classifier = Naive Bayes, Score (test, accuracy) = 35.81, Training time = 68.10 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Logistic Regression, Score (test, accuracy) = 69.08
